In [1]:
# !pip install ogame

# https://github.com/alaingilbert/pyogame/wiki/Code-Snippets

import os
import sys
module_path = os.path.abspath(os.path.join('../pyogame'))
if module_path not in sys.path:
    sys.path.append(module_path)

from ogame import OGame, constants
import time
import re
from bs4 import BeautifulSoup

import json


# to request esp reports
def BeautifulSoup4(response):
    parsed = BeautifulSoup(response, features="html5lib")

    def find_partial(**kwargs):
        for key, value in kwargs.items():
            kwargs[key] = re.compile(value)
        return parsed.find(**kwargs)

    def find_all_partial(**kwargs):
        for key, value in kwargs.items():
            kwargs[key] = re.compile(value)
        return parsed.find_all(**kwargs)

    parsed.find_partial = find_partial
    parsed.find_all_partial = find_all_partial
    return parsed


def get_reports():
    
    response = o.session.get(
        url=o.index_php,
        params={'page': 'messages',
                'tab': 20,
                'ajax': 1}
    ).text
    bs4 = BeautifulSoup4(response)
    report_links = [
        link['href']
        for link in bs4.find_all_partial(href='page=messages&messageId')
    ]
    
    data = []
    for link in report_links:
        response = o.session.get(link).text
        bs4 = BeautifulSoup4(response)
        data.append({'link': link, 'text': bs4})
        
    return data


def clean_reports(data):
    
    for report in data:
        rep_res = [_.text for _ in report['text'].find_all('span', class_="res_value")]
        rep_coord = report['text'].find('span', class_="msg_title new blue_txt").text
        try:
            rep_def = [_.text for _ in report['text'].find('ul', attrs={"data-type": "defense"}).find('li').find_all('span')]
        except:
            rep_def = None
        report['results'] = {
            'res': rep_res,
            'rep_coord': rep_coord,
            'rep_def': rep_def
        }
        
    return [_['results'] for _ in data]


### espionage nearby inactives ###

# Iterate thru galaxies
# Cut off after 10
# Check the reports, append data (cleaned)
# Continue with other 10, until done

def send_esp(o, h, r):

    data = []

    s = h-r
    e = h+r
    if e > 499:
        e = 499

    counter = 0
    for galaxy in range(s, e):
        for planet in o.galaxy(constants.coordinates(1, galaxy)):

            # stop, check reports
            if counter >= 9:
                temp_data = get_reports()
                temp_data = clean_reports(temp_data)
                data.extend(temp_data)
                counter = 0
                time.sleep(5)

            # send the probes
            if 'inactive' in planet.status and 'vacation' not in planet.status:
                send = o.send_fleet(
                    constants.mission.spy, 
                    o.planet_ids()[0], 
                    planet.position, 
                    [constants.ships.espionage_probe(1)]
                )
                counter+=1
                print(galaxy, planet.status, planet.name, planet.rank, send)
                
    return data


# find how many res you can get:
def clean_esp(data):
    coords_and_res = []
    for _ in data:
        if _['rep_def'] == None:
            res_clean = [int(__.replace('.', '')) for __ in _['res'][:3]]
            coords_and_res.append([sum(res_clean), [int(_) for _ in re.search(r'[0-9:]{1,}', _['rep_coord']).group(0).split(":")]])

    # backup
    json.dump(sorted(coords_and_res, reverse=True), open('to_raid.json', 'w'))
    
    return coords_and_res


def allocate_ships(coord_and_res, lcs, scs):
    
    coords = coord_and_res[1]
    
    if lcs > 0:
        if coord_and_res[0]/2 > 25000:
            n = int(coord_and_res[0]/2/25000)
            to_send = constants.ships.large_transporter(n)
            lcs -= n
            
        else:
            n = 1 
        to_send = constants.ships.large_transporter(n)
        # lcs -= n
        res_dep = n*25000
    
    elif lcs < 1 and scs > 0:
        n = int(coord_and_res[0]/2/5000)
        to_send = constants.ships.small_transporter(n)
        # scs -= n
        res_dep = n*5000
        
    else:
        return

    send = o.send_fleet(
        constants.mission.attack,
        o.planet_ids()[0],
        (coords[0], coords[1], coords[2], 1),
        [to_send]
    )
    
    if send:
        if to_send == constants.ships.large_transporter(n):
            lcs -= n
        if to_send == constants.ships.small_transporter(n):
            scs -= n
    else:
        return
    
    print(coord_and_res, to_send, send)
    print('remaining large:', lcs, 'remaining small:', scs)
    
    res_remaining = coord_and_res[0] - res_dep
    
    return lcs, scs, res_remaining

### waveraiding ###
def run_wave_raid(o, coords_and_res, i):
    
    # get ships
    ships_ = o.ships(o.planet_ids()[0])

    # deploy ships
    lcs = ships_.large_transporter.amount
    scs = ships_.small_transporter.amount

    for _ in sorted(coords_and_res, reverse=True)[i:]:

        lcs, scs, res_remain = allocate_ships(_, lcs, scs)
        if res_remain/2 > 25000:
            time.sleep(2)
            lcs, scs, res_remain = allocate_ships([res_remain, _[1]], lcs, scs)

In [2]:
o = OGame(
    universe="Ymir",
    username="samchakerian@gmail.com",
    password="$R3e@W1q1q"
)
data = send_esp(o, 447, 30)
coords_and_res = clean_esp(data)
run_wave_raid(o, coords_and_res, 0)

417 ['inactive'] Homeworld 1284 True
418 ['inactive'] Homeworld 1667 True
419 ['inactive'] Homeworld 1803 True
419 ['inactive'] Sybernia 1111 True
420 ['inactive'] Homeworld 1124 True
421 ['inactive'] Homeworld 1662 True
422 ['inactive'] Homeworld 1279 True
423 ['inactive'] Homeworld 1344 True
424 ['inactive'] Homeworld 1627 True
425 ['inactive'] Homeworld 1380 True
425 ['inactive'] Homeworld 1817 True
427 ['inactive'] Bones 864 True
427 ['inactive'] Colony 864 True
430 ['inactive'] Homeworld 1334 True
431 ['inactive'] Ymirs Tears 1700 True
431 ['inactive'] Colony 878 True
431 ['inactive'] Alpha 878 True
431 ['inactive'] Homeworld 1591 True
432 ['inactive'] Homeworld 1265 True
432 ['inactive'] Homeworld 1663 True
433 ['inactive'] Homeworld 1047 True
433 ['inactive'] Homeworld 1374 True
433 ['inactive'] Homeworld 1540 True
434 ['inactive'] Homeworld 1592 True
434 ['inactive'] Homeworld 1013 True
435 ['inactive'] Homeworld 1278 True
436 ['inactive'] Homeworld 1466 True
437 ['inactive'] H

TypeError: cannot unpack non-iterable NoneType object

In [5]:
# get ships
ships_ = o.ships(o.planet_ids()[0])

# deploy ships
lcs = ships_.large_transporter.amount
scs = ships_.small_transporter.amount

lcs, scs

(0, 34)

In [7]:
o

In [12]:
len(o.fleet())

11

In [11]:
for fleet in o.fleet():
    print(
        fleet.mission, fleet.arrival, fleet.destination
    )

1 False 2021-06-26 14:22:27 [1, 463, 10, 1]
1 False 2021-06-26 14:24:27 [1, 425, 4, 1]
1 False 2021-06-26 14:27:06 [1, 444, 12, 1]
1 False 2021-06-26 14:28:14 [1, 452, 6, 1]
1 False 2021-06-26 14:28:12 [1, 444, 12, 1]
1 False 2021-06-26 14:29:20 [1, 452, 6, 1]
1 False 2021-06-26 14:31:55 [1, 459, 12, 1]
1 False 2021-06-26 14:32:29 [1, 434, 12, 1]
1 False 2021-06-26 14:35:27 [1, 466, 12, 1]
1 False 2021-06-26 14:38:05 [1, 425, 4, 1]
1 False 2021-06-26 14:39:26 [1, 422, 12, 1]


In [7]:
# # reread
# # coords_and_res = json.load(open('to_raid.json', 'r'))
# sorted(coords_and_res, reverse=True)

In [8]:
# ## send raids ###

# # get ships
# ships_ = o.ships(o.planet_ids()[0])

# # deploy ships
# lcs = ships_.large_transporter.amount
# scs = ships_.small_transporter.amount

# for _ in sorted(coords_and_res, reverse=True)[:]:

#     coords = _[1]
    
#     if lcs > 0:
#         if _[0]/2 > 25000:
#             n = int(_[0]/2/25000)
#         else:
#             n = 1 
#         to_send = constants.ships.large_transporter(n)
#         lcs -= n
    
#     elif lcs < 1 and scs > 0:
#         n = int(_[0]/2/5000)
#         to_send = constants.ships.small_transporter(n)
#         scs -= n
        
#     else:
#         break

#     send = o.send_fleet(
#         constants.mission.attack,
#         o.planet_ids()[0],
#         (coords[0], coords[1], coords[2], 1),
#         [to_send]
#     )
#     print(_, to_send, send)
#     print('remaining large:', lcs, 'remaining small:', scs)

In [9]:
res = o.research()
res.computer.level

8

In [ ]:
o.build(constants.research.computer, id=o.planet_ids()[0])

In [ ]:
o.relogin()

In [ ]:
id = o.planet_ids()[0]
supply = o.supply(id)
res = o.resources(id)

# check bldg construction
print('construction:', {
    'sp': supply.solar_plant.in_construction,
    'mm': supply.metal_mine.in_construction,
    'cm': supply.crystal_mine.in_construction,
    'dm': supply.deuterium_mine.in_construction
})

# is possible
print('possible:', {
    'sp': supply.solar_plant.is_possible,
    'mm': supply.metal_mine.is_possible,
    'cm': supply.crystal_mine.is_possible,
    'dm': supply.deuterium_mine.is_possible
})

# check resources
print('resources:', {
    'sp': res.energy,
    'mm': res.metal,
    'cm': res.crystal,
    'dm': res.deuterium
})

# check levels
print('levels:', {
    'sp': supply.solar_plant.level,
    'mm': supply.metal_mine.level,
    'cm': supply.crystal_mine.level,
    'dm': supply.deuterium_mine.level
})

In [ ]:
### Iterate thru outer worlds, build const ###

